# Megaline

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

In [1]:
#Importar bibliotecas
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
#Abrir y examinar los datos
df = pd.read_csv("/datasets/users_behavior.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Se observa que el dataset contiene 3214 observaciones sin datos ausentes y con los tipos de datos adecuados para cada columna.

In [3]:
#Segmentar los datos en train, validation y test
df_train, df_test = train_test_split(df, test_size=0.25, random_state=12345)
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)

In [4]:
#Separar features y target
train_features = df_train.drop("is_ultra", axis=1)
train_target = df_train["is_ultra"]

valid_features = df_valid.drop("is_ultra", axis=1)
valid_target = df_valid["is_ultra"]

test_features = df_test.drop("is_ultra", axis=1)
test_target = df_test["is_ultra"]

print(train_features.shape)
print(train_target.shape)
print(valid_features.shape)
print(valid_target.shape)
print(test_features.shape)
print(test_target.shape)

(1807, 4)
(1807,)
(603, 4)
(603,)
(804, 4)
(804,)


In [5]:
#DecisionTreeClassifier
best_model = None
best_accuracy = 0.75
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(train_features, train_target)

    valid_predictions = model.predict(valid_features)
    test_predictions = model.predict(test_features)

    valid_accuracy = accuracy_score(valid_target, valid_predictions)
    test_accuracy = accuracy_score(test_target, test_predictions)

    print('depth=', depth)
    print('valid_accuray =', valid_accuracy)
    print('test_accuray =', test_accuracy)

    if test_accuracy > best_accuracy:
        best_model = model
        best_accuracy = test_accuracy
        best_depth = depth

print(f"accuracy del mejor modelo en el conjunto de prueba (max_depth = {best_depth}): {best_accuracy}")

depth= 1
valid_accuray = 0.7495854063018242
test_accuray = 0.75
depth= 2
valid_accuray = 0.7761194029850746
test_accuray = 0.7860696517412935
depth= 3
valid_accuray = 0.7943615257048093
test_accuray = 0.7910447761194029
depth= 4
valid_accuray = 0.7893864013266998
test_accuray = 0.7898009950248757
depth= 5
valid_accuray = 0.7877280265339967
test_accuray = 0.7873134328358209
depth= 6
valid_accuray = 0.7910447761194029
test_accuray = 0.7898009950248757
depth= 7
valid_accuray = 0.7827529021558872
test_accuray = 0.7935323383084577
depth= 8
valid_accuray = 0.7910447761194029
test_accuray = 0.7922885572139303
depth= 9
valid_accuray = 0.7744610281923715
test_accuray = 0.7873134328358209
depth= 10
valid_accuray = 0.7844112769485904
test_accuray = 0.7947761194029851
accuracy del mejor modelo en el conjunto de prueba (max_depth = 10): 0.7947761194029851


Se realizó una iteración para encontrar la profundidad adecuada del arbol de decisión encontrando que el mejor modelo corresponde a un árbol con max_depth igual a 10 con un accuracy score de 0.79

In [6]:
#RandomForestClassifier
best_model = None
best_est = 0
best_depth = 0
best_accuracy = 0.75

for est in range(10, 50, 10):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(train_features, train_target)

        valid_predictions = model.predict(valid_features)
        test_predictions = model.predict(test_features)

        valid_accuracy = accuracy_score(valid_target, valid_predictions)
        test_accuracy = accuracy_score(test_target, test_predictions)

        if test_accuracy > best_accuracy:
            best_est = est
            best_depth = depth
            best_accuracy = test_accuracy

print("La exactitud del mejor modelo en el conjunto de prueba (n_estimators = {}, max_depth = {}): {}".format(best_est, best_depth, best_accuracy))

La exactitud del mejor modelo en el conjunto de prueba (n_estimators = 20, max_depth = 10): 0.8084577114427861


Se iteró un modelo RandomForestClassifier a fin de encontrar el número óptimo de árboles y la profundidad de los mismos, obteniendo como resultado que el mejor modelo en el conjunto de prueba tiene un accuracy score de 0.8084 con n_estimators igual a 20 y max_depth igual a 10.

In [7]:
#LogisticRegression
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(train_features, train_target)
test_predictions = model.predict(test_features)
test_accuracy = accuracy_score(test_target, test_predictions)
print('accuracy del modelo de regresión logística es', test_accuracy)

accuracy del modelo de regresión logística es 0.7562189054726368


In [8]:
#Prueba de cordura
model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=10)
model.fit(valid_features[:10], valid_target[:10])
print(model.score(valid_features[:10], valid_target[:10]))

1.0


Se observa que no existe un sobreajuste del modelo ya que la prueba de cordura en una muestra pequeña determina un score de exactitud de 1

# Conclusiones
- El mejor modelo es el Random Forest con un accuracy score de 0.81 que supera el umbralde exactitud propuesto de 0.75. Se puede interpretar que el modelo predice con más del 80% de exactitud si se debe recomendar un plan Smart o Ultra.
- No existe un sobreajuste del modelo, ya que se tiene un accuracy score de 1 para una muestra de 10 datos.